# LSTM 101
Forecasting with LSTM.
Forecast steam usage based on weather.

In [1]:
DATAPATH=''
try:
    # On Google Drive, set path to my drive / data directory.
    from google.colab import drive
    IN_COLAB = True
    PATH='/content/drive/'
    drive.mount(PATH)
    DATAPATH=PATH+'My Drive/data/'  # must end in "/"
except:
    # On home computer, set path to local data directory.
    IN_COLAB = False
    DATAPATH='data/'  # must end in "/"

ZIP_FILE='BuildingData.zip'
ZIP_PATH = DATAPATH+ZIP_FILE
STEAM_FILE='steam.csv'
WEATHER_FILE='weather.csv'
MODEL_FILE='Model'  # will be used later to save models

In [2]:
from os import listdir
import csv
from zipfile import ZipFile
import numpy as np
import pandas as pd
from scipy import stats  # mode

from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
from matplotlib import colors
mycmap = colors.ListedColormap(['red','blue'])  # list color for label 0 then 1
np.set_printoptions(precision=2)

In [3]:
def read_zip_to_panda(zip_filename,csv_filename):
    zip_handle = ZipFile(zip_filename)
    csv_handle = zip_handle.open(csv_filename)
    panda = pd.read_csv(csv_handle)
    return panda
def fix_date_type(panda):
    # Convert the given timestamp column to the pandas datetime data type.
    panda['timestamp'] = pd.to_datetime(panda['timestamp'], infer_datetime_format = True)
    indexed = panda.set_index(['timestamp'])
    return indexed
def get_site_timeseries(panda,site):
    # Assume the panda dataframe has a datetime column.
    # (If not, call fix_date_type() before this.)
    # Extract the timeseries for one site.
    # Convert the datetime column to a DatetimeIndex.
    site_df = panda[panda['site_id']==site]
    temp_col = site_df['date']
    temp_val = temp_col.values
    temp_ndx = pd.DatetimeIndex(temp_val)
    dropped = site_df.drop('date',axis=1)
    panda = dropped.set_index(temp_ndx)
    return panda

In [4]:
SITE = 'Eagle'
METER = 'steam'
BLDG = 'Eagle_education_Peter'   # one example
STEPS_HISTORY = 24*1  # 1 days
STEPS_FUTURE = 24*1    # 1 day
PREDICTOR_VARIABLE = 'airTemperature'  # for starters
PREDICTED_VARIABLE = 'airTemperature'  # for starters

In [5]:
wet_df = read_zip_to_panda(ZIP_PATH,WEATHER_FILE)
wet_df = fix_date_type(wet_df)
stm_df = read_zip_to_panda(ZIP_PATH,STEAM_FILE)
stm_df = fix_date_type(stm_df)
site_specific_weather = wet_df.loc[wet_df['site_id'] == SITE]
all_buildings = [x for x in stm_df.columns if x.startswith(SITE)]

In [6]:
# For smoothing the 24 hour cycle, we do not want exponential smoothing.
def smooth(df):
    # This alternate method samples down to 1/24 sample rate
    # df.resample("24H").mean() 
    # Note the first 23 measurements get set to Nan.
    smoothed=df.rolling(window=24).mean()
    smoothed=smoothed[24:]
    return smoothed

# Correlation is low when buildings have many NaN and 0 meter readings.
# We will ignore buildings that have >max bad meter readings.
def is_usable_column(df,column_name):
    MAX_BAD = 500 
    bad = df[column_name].isin([0]).sum()
    return bad<=MAX_BAD

def prepare_for_learning(df):
    # This is very slow. Is there a faster way?
    # X = df.drop(METER,axis=1) # this would use all predictors
    X=[]
    y=[]
    predictor_series = df[PREDICTOR_VARIABLE]
    predicted_series = df[PREDICTED_VARIABLE]
    for i in range(STEPS_HISTORY,len(df)-STEPS_FUTURE):
        one_predictor = predictor_series[i-STEPS_HISTORY:i]
        one_predicted = predicted_series[i:i+STEPS_FUTURE]
        X.append(one_predictor.to_frame())
        y.append(one_predicted.to_frame())
    return X,y  # both are list of dataframe


In [15]:
cors = []
# Test on only Peter just during code development
for BLDG in ['Eagle_education_Peter']:  # all_buildings:
    # Get steam usage for one building.
    bldg_specific_steam = stm_df[[BLDG]]
    # Concatenate steam usage with weather.
    one_bldg_df = pd.concat([bldg_specific_steam,site_specific_weather],axis=1)
    # Drop the site, which is constant (we selected for one site).
    one_bldg_df = one_bldg_df.drop(['site_id'],axis=1)
    # The original steam table used column name = building name.
    # We are processing one building, so rename to the column 'steam'.
    one_bldg_df = one_bldg_df.rename(columns={BLDG : METER})
    # In order to filter bad buildings, count sum of NaN + zero.
    one_bldg_df = one_bldg_df.fillna(0)
    
    # for testing only, cut down data size
    one_bldg_df = one_bldg_df[:1000]
    
    if is_usable_column(one_bldg_df,METER):
        one_bldg_df = smooth(one_bldg_df) # moving average: 24hr
        predictor,response = prepare_for_learning(one_bldg_df)
        print("First predictor")
        print(predictor[0].tail())
        print("Second predictor")
        print(predictor[1].tail())
        print("First response")
        print(response[0])
        print("Second response")
        print(response[1])
        #print("Predictor:\n",predictor[0].info())
        #print("Response:\n",response[0].info())
        if False:
            X = one_bldg_df.drop(METER,axis=1)
            y = one_bldg_df[METER]
            # Ideally, split Year1 = train, Year2 = test.
            # Some data is incomplete, so split 1st half and 2nd half.
            split = len(X)//2 
            X_train = X.iloc[0:split]
            y_train = y.iloc[0:split]
            X_test = X.iloc[split:]
            y_test = y.iloc[split:]
            linreg = LinearRegression()
            linreg.fit(X_train,y_train)
            y_pred = linreg.predict(X_test)
            # Keep a table for reporting later.
            rmse = mean_squared_error(y_test,y_pred,squared=False)
            mean = one_bldg_df[METER].mean()
            cor = one_bldg_df.corr().iloc[0][3] # corr(steam,dew_temp)
            cors.append([cor,mean,rmse,rmse/mean,BLDG])
if False:
    print("Column 1: Correlation of steam usage to dew temp.")
    print("          Using dew temp as leading weather correlate.")
    print("Column 2: Mean steam usage.")
    print("          Using mean to help understand the RMSE.")
    print("Column 3: RMSE of LinearRegression(X=Weather, y=SteamUsage).")
    print("Column 4: RMSE/mean normalized to help understand RMSE.")
    print("Column 5: Building.")
    for cor in sorted(cors):
        print("%7.4f %10.2f %10.2f %5.2f   %s"%(cor[0],cor[1],cor[2],cor[3],cor[4]))    

First predictor
                     airTemperature
timestamp                          
2016-01-02 19:00:00        1.691667
2016-01-02 20:00:00        1.645833
2016-01-02 21:00:00        1.600000
2016-01-02 22:00:00        1.508333
2016-01-02 23:00:00        1.458333
Second predictor
                     airTemperature
timestamp                          
2016-01-02 20:00:00        1.645833
2016-01-02 21:00:00        1.600000
2016-01-02 22:00:00        1.508333
2016-01-02 23:00:00        1.458333
2016-01-03 00:00:00        1.412500
First response
                     airTemperature
timestamp                          
2016-01-03 00:00:00        1.412500
2016-01-03 01:00:00        1.320833
2016-01-03 02:00:00        1.204167
2016-01-03 03:00:00        1.062500
2016-01-03 04:00:00        0.900000
2016-01-03 05:00:00        0.829167
2016-01-03 06:00:00        0.687500
2016-01-03 07:00:00        0.595833
2016-01-03 08:00:00        0.525000
2016-01-03 09:00:00        0.525000
2016-01-03 10:00

## Useful Links

Jason Brownlee  
https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
